In [1]:
from lerobot.datasets.lerobot_dataset import LeRobotDataset

/home/bfl3/miniconda3/envs/lerobot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = LeRobotDataset("/home/bfl3/bfl_works/new_lerobot/lerobot/july5/record-test", episodes=[0])

Generating train split: 895 examples [00:00, 424409.51 examples/s]


In [4]:
dir(dataset)

['__add__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_padding_keys',
 '_get_image_file_path',
 '_get_query_indices',
 '_get_query_timestamps',
 '_is_protocol',
 '_query_hf_dataset',
 '_query_videos',
 '_save_episode_table',
 '_save_image',
 '_wait_image_writer',
 'add_frame',
 'clear_episode_buffer',
 'create',
 'create_episode_buffer',
 'create_hf_dataset',
 'delta_indices',
 'delta_timestamps',
 'download_episodes',
 'encode_episode_videos',
 'encode_videos',
 'episode_buffer',
 'episode_data_index',
 'episodes',
 'features',
 'fps',
 

In [8]:
dataset.meta


LeRobotDatasetMetadata({
    Repository ID: '/home/bfl3/bfl_works/new_lerobot/lerobot/july5/record-test',
    Total episodes: '50',
    Total frames: '44893',
    Features: '['action', 'observation.state', 'observation.images.front', 'observation.images.left', 'timestamp', 'frame_index', 'episode_index', 'index', 'task_index']',
})',

In [1]:
#!/usr/bin/env python

# Copyright 2024 The HuggingFace Inc. team. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import argparse
import time

import gymnasium as gym
import numpy as np

import gym_hil  # noqa: F401


def main():
    parser = argparse.ArgumentParser(description="Control Franka robot interactively")
    parser.add_argument("--step-size", type=float, default=0.01, help="Step size for movement in meters")
    parser.add_argument(
        "--render-mode", type=str, default="human", choices=["human", "rgb_array"], help="Rendering mode"
    )
    parser.add_argument("--use-keyboard", action="store_true", help="Use keyboard control")
    parser.add_argument(
        "--reset-delay",
        type=float,
        default=2.0,
        help="Delay in seconds when resetting the environment (0.0 means no delay)",
    )
    parser.add_argument(
        "--controller-config", type=str, default=None, help="Path to controller configuration JSON file"
    )
    args = parser.parse_args()

    # Create Franka environment - Use base environment first to debug
    env = gym.make(
        "gym_hil/PandaPickCubeBase-v0",  # Use the base environment for debugging
        render_mode=args.render_mode,
        image_obs=True,
    )

    # Print observation space for debugging
    print("Observation space:", env.observation_space)

    # Reset and check observation structure
    obs, _ = env.reset()
    print("Observation keys:", list(obs.keys()))
    if "pixels" in obs:
        print("Pixels keys:", list(obs["pixels"].keys()))

    # Now try with the wrapped version
    print("\nTrying wrapped environment...")
    env_id = (
        "gym_hil/PandaArrangeBoxesKeyboard-v0" if args.use_keyboard else "gym_hil/PandaArrangeBoxesGamepad-v0"
    )
    env = gym.make(
        env_id,
        render_mode=args.render_mode,
        image_obs=True,
        use_gamepad=not args.use_keyboard,
        max_episode_steps=1000,  # 100 seconds * 10Hz
    )

    # Print observation space for the wrapped environment
    print("Wrapped observation space:", env.observation_space)

    # Reset and check wrapped observation structure
    obs, _ = env.reset()
    print("Wrapped observation keys:", list(obs.keys()))

    # Reset environment
    obs, _ = env.reset()
    dummy_action = np.zeros(4, dtype=np.float32)
    # This ensures the "stay gripper" action is set when the intervention button is not pressed
    dummy_action[-1] = 1

    try:
        while True:
            # Step the environment
            obs, reward, terminated, truncated, info = env.step(dummy_action)

            # Print some feedback
            if info.get("succeed", False):
                print("\nSuccess! Block has been picked up.")

            # If auto-reset is disabled, manually reset when episode ends
            if terminated or truncated:
                print("Episode ended, resetting environment")
                obs, _ = env.reset()

            # Add a small delay to control update rate
            time.sleep(0.05)

    except KeyboardInterrupt:
        print("Interrupted by user")
    finally:
        env.close()
        print("Session ended")


if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] [--step-size STEP_SIZE]
                             [--render-mode {human,rgb_array}]
                             [--use-keyboard] [--reset-delay RESET_DELAY]
                             [--controller-config CONTROLLER_CONFIG]
ipykernel_launcher.py: error: unrecognized arguments: --f=/run/user/1000/jupyter/runtime/kernel-v3d156e3070c13d45a1a60c7dab49543b7b1cff72f.json


SystemExit: 2

/home/bfl3/miniconda3/envs/lerobot/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
